In [1]:
from pathlib import Path
parent_dir = str(Path.cwd().parent)
%cd $parent_dir

/Users/tomtalpir/Random/pyERGM


In [2]:
from utils import *
from ergm import ERGM, BruteForceERGM
from metrics import *

import pandas as pd
import numpy as np

In [25]:
n = 4
p = 0.25
is_directed = False

np.random.seed(98736451)
num_pos_connect = n * (n - 1)

if not is_directed:
    num_pos_connect //= 2

ground_truth_num_edges = round(num_pos_connect * p)
ground_truth_p = ground_truth_num_edges / num_pos_connect
ground_truth_theta = np.array([np.log(ground_truth_p / (1 - ground_truth_p))])

print(f"Ground truth number of edges - {ground_truth_num_edges}")
print(f"Ground truth p - {ground_truth_p}")
print(f"Ground truth theta - {ground_truth_theta}")

adj_mat_no_diag = np.zeros(num_pos_connect)
on_indices = np.random.choice(num_pos_connect, size=ground_truth_num_edges, replace=False).astype(int)
adj_mat_no_diag[on_indices] = 1
adj_mat = np.zeros((n, n))

if not is_directed:
    upper_triangle_indices = np.triu_indices(n, k=1)
    adj_mat[upper_triangle_indices] = adj_mat_no_diag
    lower_triangle_indices_aligned = (upper_triangle_indices[1], upper_triangle_indices[0])
    adj_mat[lower_triangle_indices_aligned] = adj_mat_no_diag
else:
    adj_mat[~np.eye(n, dtype=bool)] = adj_mat_no_diag

print(f"W = ")
print(adj_mat)

model = BruteForceERGM(n, MetricsCollection([NumberOfEdges()], is_directed=is_directed), is_directed=is_directed)
model.fit(adj_mat)
print(f"fit theta: {model._thetas}")
print(model._normalization_factor)

Ground truth number of edges - 2
Ground truth p - 0.3333333333333333
Ground truth theta - [-0.69314718]
W = 
[[0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
optimization started
iteration: 1, time from start training: 0.02153491973876953 log likelihood: -4.1597930984858085
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 4.1597930984858085
        x: [ 3.483e-02]
      nit: 1
      jac: [ 1.052e+00]
 hess_inv: [[ 6.711e-01]]
     nfev: 49
     njev: 38
fit theta: [0.0348341]
147.9394974344226


In [24]:
n = 4
metrics_calc = MetricsCollection([NumberOfEdges()], is_directed=is_directed)
fitted_model = ERGM(n, metrics_calc, is_directed=is_directed, n_networks_for_grad_estimation=10, n_mcmc_steps=200)
fitted_model.fit(adj_mat)

optimization started
iteration: 1, time from start training: 0.17136383056640625 log likelihood: -4.222169073570396
	Optimization result:
	Theta: [-0.34805477]
	Normalization factor: 51.74318593817907
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 4.222169073570396
        x: [-3.481e-01]
      nit: 1
      jac: [ 2.000e-01]
 hess_inv: [[ 6.250e-02]]
     nfev: 28
     njev: 12


In [ ]:

model = BruteForceERGM(n, MetricsCollection([NumberOfEdges()], is_directed=is_directed), is_directed=is_directed)
model.fit(adj_mat)

# print(f"ground truth theta: {ground_truth_theta}")
# print(f"fit theta: {model._thetas}")

# for t_model, t_ground_truth in zip(model._thetas, ground_truth_theta):
#     self.assertAlmostEqual(t_model, t_ground_truth, places=5)

# non_synapses_indices = np.where(adj_mat_no_diag == 0)[0]
# prediction = ground_truth_p * np.ones(adj_mat_no_diag.size)
# prediction[non_synapses_indices] = 1 - ground_truth_p
# true_log_like = np.log(prediction).sum()
# print(f"true log likelihood: {true_log_like}")

# model_with_true_theta = BruteForceERGM(n, MetricsCollection([NumberOfEdges()], is_directed=is_directed),
#                                         initial_thetas=np.array(ground_truth_theta), is_directed=is_directed)

# ground_truth_model_log_like = np.log(model_with_true_theta.calculate_weight(adj_mat)) - np.log(
#     model_with_true_theta._normalization_factor)

# print(f"model with true theta log like: {ground_truth_model_log_like}")